# Training your own model

This notebook will walk you through training your own model using [DeCLUTR](https://github.com/JohnGiorgi/DeCLUTR).

## 🔧 Install the prerequisites

In [ ]:
!pip install git+https://github.com/JohnGiorgi/DeCLUTR.git

## 📖 Preparing a dataset


A dataset is simply a file containing one item of text (a document, a scientific paper, etc.) per line. For demonstration purposes, we have provided a script that will download the [WikiText-103](https://www.salesforce.com/products/einstein/ai-research/the-wikitext-dependency-language-modeling-dataset/) dataset and format it for training with our method.

The only "gotcha" is that each piece of text needs to be long enough so that we can sample spans from it. In general, you should collect documents of a minimum length according to the following:

```python
min_length = num_anchors * max_span_len * 2
```

In our paper, we set `num_anchors=2` and `max_span_len=512`, so we require documents of `min_length=2048`. We simply need to provide this value as an argument when running the script:

In [ ]:
import os

train_data_path = "wikitext_103/train.txt"
min_length = 2048

!wget -nc https://raw.githubusercontent.com/JohnGiorgi/DeCLUTR/master/scripts/preprocess_wikitext_103.py
!python preprocess_wikitext_103.py $train_data_path --min-length $min_length

Lets confirm that our dataset looks as expected.

In [ ]:
!wc -l $train_data_path  # This should be approximately 17.8K lines

In [ ]:
!head -n 1 $train_data_path  # This should be a single Wikipedia entry

## 🏃 Training the model

Once you have collected the dataset, you can easily initiate a training session with the `allennlp train` command. An experiment is configured using a [Jsonnet](https://jsonnet.org/) config file. Lets take a look at the config for the DeCLUTR-small model presented in [our paper](https://arxiv.org/abs/2006.03659):

In [ ]:
!wget -nc https://raw.githubusercontent.com/JohnGiorgi/DeCLUTR/master/training_config/declutr_small.jsonnet
with open("declutr_small.jsonnet", "r") as f:
    print(f.read())


The only thing to configure is the path to the training set (`train_data_path`), which can be passed to `allennlp train` via the `--overrides` argument (but you can also provide it in your config file directly, if you prefer):

In [ ]:
overrides = (
    f"{{'train_data_path': '{train_data_path}', "
    # lower the batch size to be able to train on Colab GPUs
    "'data_loader.batch_size': 2, "
    # training examples / batch size. Not required, but gives us a more informative progress bar during training
    "'data_loader.batches_per_epoch': 8912}"
)

In [ ]:
overrides

In [ ]:
!allennlp train "declutr_small.jsonnet" \
    --serialization-dir "output" \
    --overrides "$overrides" \
    --include-package "declutr" \
    -f

### 🤗 Exporting a trained model to HuggingFace Transformers

We have provided a simple script to export a trained model so that it can be loaded with [Hugging Face Transformers](https://github.com/huggingface/transformers)

In [ ]:
!wget -nc https://github.com/JohnGiorgi/DeCLUTR/blob/master/scripts/save_pretrained_hf.py
!python save_pretrained_hf.py --archive-file "output" --save-directory "output_transformers"

The model, saved to `--save-directory`, can then be loaded using the Hugging Face Transformers library

> See the [embedding notebook](https://colab.research.google.com/github/JohnGiorgi/DeCLUTR/blob/master/notebooks/embedding.ipynb) for more details on using trained models.

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
  
tokenizer = AutoTokenizer.from_pretrained("output_transformers")
model = AutoModel.from_pretrained("output_transformers")

> If you would like to upload your model to the Hugging Face model repository, follow the instructions [here](https://huggingface.co/transformers/model_sharing.html).

## ♻️ Conclusion

That's it! In this notebook, we covered how to collect data for training the model, and specifically how _long_ that text needs to be. We then briefly covered configuring and running a training session. Please see [our paper](https://arxiv.org/abs/2006.03659) and [repo](https://github.com/JohnGiorgi/DeCLUTR) for more details, and don't hesitate to open an issue if you have any trouble!